In [1]:
# Utils
import json
import os

# Setting logging level
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or '3' to additionally suppress all warnings

# TensorFlow Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, MaxPooling2D, BatchNormalization, Conv2D, Dense, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model

# In this notebook, I train four different Keras Sequential models using a cleaned dataset. The models, named model_1_concept, model_2_best, model_3_best_nodropout, and model_4_best_nodropout_nobatchn, 
# were developed through experimentation and adjustments based on the initial concept and literature recommendations for convolutional neural networks (CNN).

# The code begins with importing necessary modules and creating ImageDataGenerators and flow_from_directory functions for both the training and test sets. 
# The model architectures are specified in JSON files, which are read by the code to iteratively build each Keras model. This allows easy iteration over different models and manual adjustments to optimizers and color modes (RGB or grayscale). 
# The training history for each model is saved, enabling tracking and visualization of performance and learning. 
# Additionally, model architectures are saved in the "diagram" folder, checkpoint files are saved on improved epochs, and final models are saved as h5 files.

# Examples of saved file formats include:
#     History: model_1_concept_adam_grayscale_32_augment_history.json
#     Final model (not included on Git due to size): model_2_best_rmsprop_rgb_512_augment_final.h5
#     Checkpoint: model_3_best_nodropout_rmsprop_rgb_512_augment_cpt.h5

# For practicality, only the best-performing model is saved in a dedicated "models/best_model" folder, including both the architecture (JSON file) and weights (h5 file).

In [2]:
# Image Specifications
folder_path = "../dataset/MMAFEDB/"
image_dimension = (48, 48)
image_depth = 3
image_color_mode = 'rgb'
batch_size = 512
augment = 'augment'

# Data Augmentation for Training Set
datagen_train = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only rescaling for Validation Set
datagen_val = ImageDataGenerator(
    rescale=1./255
)

# Training flow_from_directory
train_set = datagen_train.flow_from_directory(
    folder_path + "train",
    target_size=image_dimension,
    color_mode=image_color_mode,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Testing flow_from_directory
test_set = datagen_val.flow_from_directory(
    folder_path + "test",
    target_size=image_dimension,
    color_mode=image_color_mode,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Get the total number of images in the training set
total_images_train = train_set.n
print("Total images in training set:", total_images_train)
# Get the total number of images in the test set
total_images_test = test_set.n
print("Total images in test set:", total_images_test)

Found 92968 images belonging to 7 classes.
Found 17356 images belonging to 7 classes.
Total images in training set: 92968
Total images in test set: 17356


In [3]:
# Imports
import os

# Specify the path to the folder containing your JSON files
folder_path = '../models/'

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Filter only JSON files
json_files = sorted([file for file in files if file.endswith('.json')])
print(f'Found JSON config files in models: {json_files}\n')

# Iterate through the JSON files and load them
for json_file in json_files:
    json_file_path = os.path.join(folder_path, json_file)
    
    print(f'########################################')
    print(f'Using File {json_file} to train model!')
    print(f'########################################\n')

    with open(json_file_path,mode='r') as file:
        m = json.load(file)
        print('JSON Loaded:',m)
        print()

        # Patching first layer with input to be aligned with the image_depth (e.g. if using grayscale vs rgb)
        m['Conv2D_1']['input_shape'] = [48,48,image_depth]

        # Initializating the model from JSON configuration
        # Step 1 - Create Sequential model
        print('Creating Sequential Model ...')
        model = Sequential()

        # Step 2 - Iteratively add the configured layers and their parameters
        for layer_name, layer_config in m.items():
            layer_type = layer_name.split("_")[0]
            print(f"Added {layer_name}: {layer_config}")

            # Add layers based on layer type
            if layer_type == 'Conv2D':
                model.add(Conv2D(**layer_config))
            elif layer_type == 'MaxPooling2D':
                model.add(MaxPooling2D(**layer_config))
            elif layer_type == 'Flatten':
                model.add(Flatten(**layer_config))
            elif layer_type == 'Dense':
                model.add(Dense(**layer_config))
            elif layer_type == 'BatchNormalization':
                model.add(BatchNormalization(**layer_config))
            elif layer_type == 'Dropout':
                model.add(Dropout(**layer_config))
            else:
                # Handle unrecognized layer types or raise an exception
                raise ValueError(f"Unsupported layer type: {layer_type}")
        
        # Step 3 - Optimizer Values
        optimizer = {
            'adam': tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
            'rmsprop': tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9),
            'sgd': tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
        }
        print(f"\nOptimizers to select from: {list(optimizer.keys())}")
        selected_optimizer = 'rmsprop'
        print(f"\nSelected Optimizer for training: {selected_optimizer}")

        # Step 4 - Loss function
        loss = 'categorical_crossentropy'
        print(f"Loss function used: {loss}")

        # Step 5 - Metrics
        metrics = ['accuracy']
        print(f"Metrics used: [{metrics}]")
        
        # Compile the model with settings
        model.compile(
            optimizer=optimizer['adam'],
            loss=loss,
            metrics=metrics
        )
        print('The model has been compiled, showing summary below!\n')
        # Print model summary
        model.summary()

        # Save the visual diagram to an image file (e.g., PNG)
        # Model architecture remains the same, so we can store them all under one file_name for each different model (model_1_concept, model_2_best, model_3_best_nodropout ...)
        diagram_filename = folder_path + f"training/diagram/{json_file.split('.')[0]}_diagram.png"
        print(f'Saving plot_model figure of current model layout to {diagram_filename}')
        plot_model(model, to_file=diagram_filename, show_shapes=True, show_layer_names=True)

        # Start model training!
        # Define hyperparameters
        checkpoint_filename = folder_path+f"training/checkpoint/{json_file.split('.')[0]}_{selected_optimizer}_{image_color_mode}_{batch_size}_{augment}_cpt.h5"
        epochs = 50
        monitor='val_accuracy'
        mode='max'

        # Set up model checkpoint
        print('\nSetting up checkpoint file for incremental validation accuracy improvements')
        checkpoint = ModelCheckpoint(
            filepath=checkpoint_filename,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            mode=mode
        )
        callbacks_list = [checkpoint]

        # Train the model
        print(f'********************** Model training started! ********************')
        history = model.fit(
            train_set,
            validation_data=test_set,
            steps_per_epoch=total_images_train // batch_size,
            validation_steps=total_images_test // batch_size,
            epochs=epochs,
            callbacks=callbacks_list
        )
        print(f'********************** Model training finished! ********************')

        # Saving final model after train epochs have completed
        final_filename = folder_path+f"training/final/{json_file.split('.')[0]}_{selected_optimizer}_{image_color_mode}_{batch_size}_{augment}_final.h5"
        print(f'Saving final model to {final_filename}')
        model.save_weights(final_filename)

        # Saving training history to file
        history_filename = folder_path+f"training/history/{json_file.split('.')[0]}_{selected_optimizer}_{image_color_mode}_{batch_size}_{augment}_history.json"
        with open(history_filename, 'w') as history_file:
            json.dump(history.history, history_file)
            print(f'Saved model.fit training history to {history_filename}')

        # End of training loop!
        print('########################################################################################')
        print('##################### Loop has finished for current model file! ########################')
        print('########################################################################################')

Found JSON config files in models: ['model_1_concept.json', 'model_2_best.json', 'model_3_best_nodropout.json', 'model_4_best_nodroupout_nobatchn.json']

########################################
Using File model_1_concept.json to train model!
########################################

JSON Loaded: {'Conv2D_1': {'filters': 32, 'kernel_size': 3, 'activation': 'relu', 'input_shape': [48, 48, 3]}, 'MaxPooling2D_1': {'pool_size': 2}, 'Conv2D_3': {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'}, 'Conv2D_4': {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'}, 'MaxPooling2D_2': {'pool_size': 2}, 'Conv2D_5': {'filters': 128, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'}, 'Conv2D_6': {'filters': 128, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'}, 'MaxPooling2D_3': {'pool_size': 2}, 'Conv2D_7': {'filters': 256, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'}, 'Conv2D_8': {'filters': 256, 'kernel_size': 3, 'activ

/home/fox/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


181/181 [==============================] - ETA: 0s - loss: 1.1933 - accuracy: 0.5742
Epoch 2: val_accuracy improved from 0.44437 to 0.45620, saving model to ../models/training/checkpoint/model_1_concept_rmsprop_rgb_512_augment_cpt.h5
181/181 [==============================] - 59s 322ms/step - loss: 1.1933 - accuracy: 0.5742 - val_loss: 1.5214 - val_accuracy: 0.4562
Epoch 3/50
181/181 [==============================] - ETA: 0s - loss: 1.1247 - accuracy: 0.5999
Epoch 3: val_accuracy improved from 0.45620 to 0.48319, saving model to ../models/training/checkpoint/model_1_concept_rmsprop_rgb_512_augment_cpt.h5
181/181 [==============================] - 59s 324ms/step - loss: 1.1247 - accuracy: 0.5999 - val_loss: 1.5444 - val_accuracy: 0.4832
Epoch 4/50
181/181 [==============================] - ETA: 0s - loss: 1.0745 - accuracy: 0.6186
Epoch 4: val_accuracy improved from 0.48319 to 0.49065, saving model to ../models/training/checkpoint/model_1_concept_rmsprop_rgb_512_augment_cpt.h5
181/181 

2023-12-29 14:28:46.982554: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


181/181 [==============================] - ETA: 0s - loss: 1.7501 - accuracy: 0.4373
Epoch 1: val_accuracy improved from -inf to 0.35500, saving model to ../models/training/checkpoint/model_2_best_rmsprop_rgb_512_augment_cpt.h5


...

181/181 [==============================] - 64s 294ms/step - loss: 1.7501 - accuracy: 0.4373 - val_loss: 1.6855 - val_accuracy: 0.3550
Epoch 2/50
181/181 [==============================] - ETA: 0s - loss: 1.3172 - accuracy: 0.5554
Epoch 2: val_accuracy improved from 0.35500 to 0.40442, saving model to ../models/training/checkpoint/model_2_best_rmsprop_rgb_512_augment_cpt.h5
181/181 [==============================] - 50s 276ms/step - loss: 1.3172 - accuracy: 0.5554 - val_loss: 1.6696 - val_accuracy: 0.4044
Epoch 3/50
181/181 [==============================] - ETA: 0s - loss: 1.1916 - accuracy: 0.6011
Epoch 3: val_accuracy improved from 0.40442 to 0.49917, saving model to ../models/training/checkpoint/model_2_best_rmsprop_rgb_512_augment_cpt.h5
181/181 [==============================] - 50s 276ms/step - loss: 1.1916 - accuracy: 0.6011 - val_loss: 1.3889 - val_accuracy: 0.4992
Epoch 4/50
181/181 [==============================] - ETA: 0s - loss: 1.1290 - accuracy: 0.6204
Epoch 4: val_accur

/home/fox/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


181/181 [==============================] - 57s 289ms/step - loss: 1.6231 - accuracy: 0.4714 - val_loss: 1.9784 - val_accuracy: 0.3182
Epoch 2/50
181/181 [==============================] - ETA: 0s - loss: 1.1654 - accuracy: 0.5870
Epoch 2: val_accuracy improved from 0.31818 to 0.44300, saving model to ../models/training/checkpoint/model_3_best_nodropout_rmsprop_rgb_512_augment_cpt.h5
181/181 [==============================] - 50s 273ms/step - loss: 1.1654 - accuracy: 0.5870 - val_loss: 1.8059 - val_accuracy: 0.4430
Epoch 3/50
181/181 [==============================] - ETA: 0s - loss: 1.0598 - accuracy: 0.6258
Epoch 3: val_accuracy improved from 0.44300 to 0.50941, saving model to ../models/training/checkpoint/model_3_best_nodropout_rmsprop_rgb_512_augment_cpt.h5
181/181 [==============================] - 50s 273ms/step - loss: 1.0598 - accuracy: 0.6258 - val_loss: 1.4420 - val_accuracy: 0.5094
Epoch 4/50
181/181 [==============================] - ETA: 0s - loss: 1.0026 - accuracy: 0.646

/home/fox/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


181/181 [==============================] - 50s 269ms/step - loss: 1.6259 - accuracy: 0.3379 - val_loss: 1.5792 - val_accuracy: 0.3503
Epoch 2/50
181/181 [==============================] - ETA: 0s - loss: 1.4579 - accuracy: 0.4438
Epoch 2: val_accuracy improved from 0.35026 to 0.45632, saving model to ../models/training/checkpoint/model_4_best_nodroupout_nobatchn_rmsprop_rgb_512_augment_cpt.h5
181/181 [==============================] - 49s 269ms/step - loss: 1.4579 - accuracy: 0.4438 - val_loss: 1.4560 - val_accuracy: 0.4563
Epoch 3/50
181/181 [==============================] - ETA: 0s - loss: 1.2206 - accuracy: 0.5610
Epoch 3: val_accuracy improved from 0.45632 to 0.49497, saving model to ../models/training/checkpoint/model_4_best_nodroupout_nobatchn_rmsprop_rgb_512_augment_cpt.h5
181/181 [==============================] - 49s 269ms/step - loss: 1.2206 - accuracy: 0.5610 - val_loss: 1.4012 - val_accuracy: 0.4950
Epoch 4/50
181/181 [==============================] - ETA: 0s - loss: 1.10